# Getting __more__ stock data

In [1]:
import numpy as np
import pandas as pd
import dill
from sqlalchemy import create_engine, inspect
from tqdm import tqdm_notebook
from alpha_vantage.timeseries import TimeSeries

So I'm going to try another API provider, [Alpha Vantage](https://www.alphavantage.co/documentation/), to see if they offer a more complete listing of end of day stock prices. The number to beat is 110 companies.

...Lets get started by importing our set of tickers

In [2]:
future_Ticks = dill.load(open('future_ticker_symbols.pkl', 'r'))

In [3]:
past_Ticks = dill.load(open('past_ticker_symbols.pkl', 'r'))

In [4]:
tickSymbs = future_Ticks.union(past_Ticks)

...and our Alpha Vantage API key, from an ignored file not pushed to my public repo

In [5]:
av_key_handle = open("alphavantage.apikey", "r")
ts = TimeSeries(key=av_key_handle.read().strip(), output_format='pandas')
av_key_handle.close()

...finally we can connect to the sqlite database and start pulling down data

In [6]:
engine = create_engine('sqlite:///capstone.db')

In [7]:
losers = []
winners = []
for ticker in tqdm_notebook(tickSymbs):
    try:
        df, meta = ts.get_daily(symbol=ticker, outputsize='full')
        df.to_sql(meta["2. Symbol"], engine, if_exists = 'replace')
    except:
        losers.append(ticker)
    else:
        winners.append(meta["2. Symbol"])
        '''print meta["2. Symbol"]
        print df.head()
        print df.tail()'''

In [9]:
print len(winners), winners
print len(losers), losers

361 ['ENTA', 'AGN', 'LIFE', 'KITE', 'BSTC', 'AAAP', 'FLML', 'SAGE', 'PAR', 'BPTH', 'AGEN', 'ESPR', 'RETA', 'IRWD', 'VNDA', 'RPTP', 'BYSI', 'ACUR', 'DEPO', 'ARIA', 'CNAT', 'ALDR', 'VRTX', 'THRX', 'HPTX', 'AGLE', 'DRRX', 'EXEL', 'FLXN', 'XBIT', 'JNCE', 'MEIP', 'ADMP', 'ADMS', 'OTIC', 'ACOR', 'CEMP', 'KMPH', 'RLYP', 'ANAB', 'TNXP', 'ITEK', 'JNJ', 'PSTI', 'SNY', 'AMAG', 'AFMD', 'GWPH', 'GBT', 'CRTX', 'CRMD', 'LPCN', 'SVRA', 'VCEL', 'PTGX', 'ADXS', 'PHMMF', 'DCTH', 'AEGR', 'TBPH', 'DVAX', 'SGEN', 'IOVA', 'BHVN', 'NVLS', 'EGRX', 'QCOR', 'AFFY', 'BCRX', 'AXSM', 'GLMD', 'CPRX', 'DBVT', 'RDUS', 'IPXL', 'NGSX', 'RMTI', 'NVS', 'RNN', 'DYAX', 'ALTH', 'ALDX', 'OPHT', 'KURA', 'CBIO', 'NVO', 'MELA', 'OPK', 'EBIO', 'HRTX', 'ONTX', 'MRUS', 'SPPI', 'AMGN', 'CRBP', 'DOR', 'WCRX', 'KTOV', 'SMMT', 'AZN', 'VRX', 'CADX', 'EPIX', 'IGXT', 'HALO', 'MBVX', 'BLRX', 'OREX', 'OPTR', 'LOXO', 'SNDX', 'CNCE', 'IMGN', 'LJPC', 'MNOV', 'GALE', 'ATRS', 'CLSD', 'HEB', 'INNL', 'CLSN', 'TXMD', 'CHRS', 'ALIM', 'GNBT', 'PLX', 

Lets check in the morning how she did, how far back can she read and what % of companies were successfully pulled?